In [1]:
import numpy as np
import pandas as pd
import pyreadr
import random
import statsmodels.formula.api as sm
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from helper.functions import fn_generate_data, fn_generate_variables, fn_generate_df_prop, fn_generate_df_matched

np.random.seed(570)
random.seed(570)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

C:\Users\imym0\Anaconda3\envs\finance\lib\site-packages\statsmodels\compat\pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
df_nsw = pyreadr.read_r('nsw.rda')['nsw']
df_nsw.describe()

,treat,age,education,black,hispanic,married,nodegree,re75,re78
count,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000
mean,0.411357,24.520776,10.267313,0.800554,0.105263,0.162050,0.779778,3042.896585,5454.635844
std,0.492421,6.625947,1.704774,0.399861,0.307105,0.368752,0.414683,5066.143382,6252.943413
min,0.000000,17.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,19.000000,9.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,0.000000,23.000000,10.000000,1.000000,0.000000,0.000000,1.000000,936.307953,3951.889038
75%,1.000000,27.000000,11.000000,1.000000,0.000000,0.000000,1.000000,3993.206970,8772.004395
max,1.000000,55.000000,16.000000,1.000000,1.000000,1.000000,1.000000,37431.660156,60307.929688


In [3]:
df_nsw.loc[df_nsw['treat'] == 1, 'data_id'] = 'LT'
df_nsw.loc[df_nsw['treat'] == 0, 'data_id'] = 'LC'

df_psid1 = pyreadr.read_r('psid_controls.rda')['psid_controls']
df_psid1 = df_psid1.drop(['re74'], axis = 1) # Drop earnings in 1974 for now
df_psid2 = pyreadr.read_r('psid_controls2.rda')['psid_controls2']
df_psid2 = df_psid2.drop(['re74'], axis = 1)
df_psid3 = pyreadr.read_r('psid_controls3.rda')['psid_controls3']
df_psid3 = df_psid3.drop(['re74'], axis = 1)
df_cps1 = pyreadr.read_r('cps_controls.rda')['cps_controls']
df_cps1 = df_cps1.drop(['re74'], axis = 1)
df_cps2 = pyreadr.read_r('cps_controls2.rda')['cps_controls2']
df_cps2 = df_cps2.drop(['re74'], axis = 1)
df_cps3 = pyreadr.read_r('cps_controls3.rda')['cps_controls3']
df_cps3 = df_cps3.drop(['re74'], axis = 1)

df = pd.concat([df_nsw, df_psid1, df_psid2, df_psid3, df_cps1, df_cps2, df_cps3])
df

,data_id,treat,age,education,black,hispanic,married,nodegree,re75,re78
0,LT,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.0,9930.045898
1,LT,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.0,3595.894043
2,LT,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,24909.449219
3,LT,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.0,7506.145996
4,LT,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.0,289.789886
...,...,...,...,...,...,...,...,...,...,...
424,CPS3,0.0,18.0,11.0,0.0,0.0,0.0,1.0,0.0,10150.500000
425,CPS3,0.0,24.0,1.0,0.0,1.0,1.0,1.0,0.0,19464.609375
426,CPS3,0.0,21.0,18.0,0.0,0.0,0.0,0.0,0.0,0.000000
427,CPS3,0.0,32.0,5.0,1.0,0.0,1.0,1.0,0.0,187.671295


In [4]:
# Difference in revenues
df['dif'] = df['re78'] - df['re75']
df

,data_id,treat,age,education,black,hispanic,married,nodegree,re75,re78,dif
0,LT,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.0,9930.045898,9930.045898
1,LT,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.0,3595.894043,3595.894043
2,LT,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,24909.449219,24909.449219
3,LT,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.0,7506.145996,7506.145996
4,LT,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.0,289.789886,289.789886
...,...,...,...,...,...,...,...,...,...,...,...
424,CPS3,0.0,18.0,11.0,0.0,0.0,0.0,1.0,0.0,10150.500000,10150.500000
425,CPS3,0.0,24.0,1.0,0.0,1.0,1.0,1.0,0.0,19464.609375,19464.609375
426,CPS3,0.0,21.0,18.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
427,CPS3,0.0,32.0,5.0,1.0,0.0,1.0,1.0,0.0,187.671295,187.671295


In [5]:
# Check what is in data_id
df.data_id.unique()

array(['LT', 'LC', 'PSID', 'PSID2', 'PSID3', 'CPS1', 'CPS2', 'CPS3'],
      dtype=object)

In [6]:
# 'True' average treatment effects
df0 = fn_generate_data(data_id = 'LC', df = df)
results = sm.ols(formula = 'dif ~ treat', data = df0).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.286
Date:                Sun, 06 Mar 2022   Prob (F-statistic):              0.131
Time:                        22:54:56   Log-Likelihood:                -7456.6
No. Observations:                 722   AIC:                         1.492e+04
Df Residuals:                     720   BIC:                         1.493e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2063.3655    359.259      5.743      0.000    1358.045    2768.686
treat        846.8883    560.142      1.512      0.131    -252.818    1946.594
==============================================================================
Omnibus:                      161.292   Durbin-Watson:                   1.743
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1522.870
Skew:                           0.709   Prob(JB):                         0.00
Kurtosis:                       9.972   Cond. No.                         2.46
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
df0['age2'] = df0['age'] ** 2

results = sm.ols(formula = 'dif ~ treat + age + age2 + education + black + hispanic + nodegree', data = df0).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1.823
Date:                Sun, 06 Mar 2022   Prob (F-statistic):             0.0799
Time:                        22:54:56   Log-Likelihood:                -7451.3
No. Observations:                 722   AIC:                         1.492e+04
Df Residuals:                     714   BIC:                         1.496e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.173e+04   4344.694      2.700      0.007    3202.218    2.03e+04
treat        819.2771    561.491      1.459      0.145    -283.094    1921.648
age         -746.4430    251.135     -2.972      0.003   -1239.495    -253.391
age2          12.3287      4.229      2.915      0.004       4.025      20.632
education     87.1087    217.554      0.400      0.689    -340.013     514.230
black         77.0276    956.036      0.081      0.936   -1799.950    1954.005
hispanic     977.7160   1253.769      0.780      0.436   -1483.799    3439.231
nodegree    -469.8266    891.073     -0.527      0.598   -2219.263    1279.610
==============================================================================
Omnibus:                      183.214   Durbin-Watson:                   1.732
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1758.888
Skew:                           0.846   Prob(JB):                         0.00
Kurtosis:                      10.457   Cond. No.                     1.21e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.21e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
results = sm.ols(formula = 'dif ~ treat + age + age2', data = df0).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     3.606
Date:                Sun, 06 Mar 2022   Prob (F-statistic):             0.0132
Time:                        22:54:56   Log-Likelihood:                -7452.3
No. Observations:                 722   AIC:                         1.491e+04
Df Residuals:                     718   BIC:                         1.493e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.184e+04   3380.152      3.503      0.000    5205.814    1.85e+04
treat        856.9110    557.678      1.537      0.125    -237.964    1951.786
age         -703.6823    241.631     -2.912      0.004   -1178.070    -229.294
age2          11.5827      4.079      2.839      0.005       3.574      19.591
==============================================================================
Omnibus:                      183.011   Durbin-Watson:                   1.749
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1741.797
Skew:                           0.847   Prob(JB):                         0.00
Kurtosis:                      10.418   Cond. No.                     9.33e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.33e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Average treatment effect on `dif` is about 820-860.

In [9]:
# Use CPS1
df1 = fn_generate_data(data_id = 'CPS1', df = df)
df1

,data_id,treat,age,education,black,hispanic,married,nodegree,re75,re78,dif
0,LT,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.000000,9930.045898,9930.045898
1,LT,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.000000,3595.894043,3595.894043
2,LT,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.000000,24909.449219,24909.449219
3,LT,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.000000,7506.145996,7506.145996
4,LT,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.000000,289.789886,289.789886
...,...,...,...,...,...,...,...,...,...,...,...
16284,CPS1,0.0,22.0,12.0,1.0,0.0,0.0,0.0,6801.435059,2757.437988,-4043.997070
16285,CPS1,0.0,20.0,12.0,1.0,0.0,1.0,0.0,11832.240234,6895.071777,-4937.168457
16286,CPS1,0.0,37.0,12.0,0.0,0.0,0.0,0.0,1559.370972,4221.865234,2662.494263
16287,CPS1,0.0,47.0,9.0,0.0,0.0,1.0,1.0,11384.660156,13671.929688,2287.269531


In [10]:
# Generate variables
X, T, Y = fn_generate_variables(data = df1, outcome = 'dif')

# Estimate propensity score by Random Forest
param_grid_p = {'n_estimators': [50, 100, 500, 1000], 'max_features': [2, 3, 4, 5]}

rfc = GridSearchCV(RandomForestClassifier(), param_grid = param_grid_p, cv = 5,
                   scoring = 'neg_mean_squared_error', return_train_score = False, verbose = 1,
                   error_score = 'raise')
rfc.fit(X, T.ravel())

phat = np.array(rfc.predict_proba(X)[:, 1], ndmin = 2).T

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [11]:
# Generate a data frame with propensity score
# The data with extremely high or low pronepsity scores are removed
df_prop = fn_generate_df_prop(df = df1, prop = phat, truncate_level = 0.01)
df_prop

,data_id,treat,age,education,black,hispanic,married,nodegree,re75,re78,dif,propensity_score,propensity_score_logit
0,LT,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.000000,9930.045898,9930.045898,0.626667,0.517943
1,LT,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.000000,3595.894043,3595.894043,0.714200,0.915871
2,LT,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.000000,24909.449219,24909.449219,0.313471,-0.783940
3,LT,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.000000,7506.145996,7506.145996,0.990000,4.595120
4,LT,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.000000,289.789886,289.789886,0.802000,1.398842
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,CPS1,0.0,17.0,9.0,1.0,0.0,0.0,1.0,0.000000,422.629791,422.629791,0.497494,-0.010024
1153,CPS1,0.0,42.0,12.0,1.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.016267,-4.102237
1154,CPS1,0.0,20.0,12.0,1.0,0.0,0.0,0.0,9737.565430,3771.157959,-5966.407471,0.075000,-2.512306
1155,CPS1,0.0,17.0,9.0,1.0,0.0,0.0,1.0,2112.581055,3039.684082,927.103027,0.141000,-1.807009


In [12]:
# Propensity score matching
df_matched = fn_generate_df_matched(df = df_prop, outcome = 'dif', n_neighbors = 1)
df_matched

,data_id,treat,age,education,black,hispanic,married,nodegree,re75,re78,dif,propensity_score,propensity_score_logit,matched_index_1,matched_outcome_1
0,LT,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.000000,9930.045898,9930.045898,0.626667,0.517943,827.0,1053.619019
1,LT,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.000000,3595.894043,3595.894043,0.714200,0.915871,827.0,1053.619019
2,LT,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.000000,24909.449219,24909.449219,0.313471,-0.783940,1108.0,1161.493042
3,LT,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.000000,7506.145996,7506.145996,0.990000,4.595120,827.0,1053.619019
4,LT,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.000000,289.789886,289.789886,0.802000,1.398842,827.0,1053.619019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,CPS1,0.0,17.0,9.0,1.0,0.0,0.0,1.0,0.000000,422.629791,422.629791,0.497494,-0.010024,128.0,1953.267944
1153,CPS1,0.0,42.0,12.0,1.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.016267,-4.102237,9.0,12418.070312
1154,CPS1,0.0,20.0,12.0,1.0,0.0,0.0,0.0,9737.565430,3771.157959,-5966.407471,0.075000,-2.512306,9.0,12418.070312
1155,CPS1,0.0,17.0,9.0,1.0,0.0,0.0,1.0,2112.581055,3039.684082,927.103027,0.141000,-1.807009,96.0,8048.603027


In [13]:
# ATET
df_matched[df_matched.treat == 1]['dif'].mean() - df_matched[df_matched.treat == 1]['matched_outcome_1'].mean()

1650.1522635640324

Much larger than the experimental data.